## 1. Library Import

In [1]:
import numpy as np
import time
import itertools

__author__ = "Marco Odehnal"
__copyright__ = "Copyright 2018"
__status__ = "Prototype"

## 2. Problem formulation

In [183]:
# The cities for the problem, taken from att48
cities = np.matrix([[1, 6734, 1453],
                    [2, 2233, 10],
                    [3, 5530, 1424],
                    [4, 401, 841],
                    [5, 3082, 1644],
                    [6, 7608, 4458],
                    [7, 7573, 3716],
                   [8, 123, 8526],
                   [9, 4356, 376],
                   [10, 7456, 236],
                    [11, 67534, 14553],
                    [2, 22353, 150],
                    [3, 55530, 14524],
                    [4, 45501, 841],
                    [5, 350852, 16544],
                    [6, 76508, 4458],
                    [7, 75573, 35716],
                   [8, 1253, 85526],
                   [9, 43556, 5376],
                   [10, 57456, 2536]                   ])

# The number of verteces 
n = np.size(cities,0)

# Adjacency matrix
# NOT OPTIMAL GENERATION, JUST FOR TESTING
M = np.matrix([[np.linalg.norm(cities[i,1:3]-cities[j,1:3]) for j in range(n)] for i in range(n)])

## 3. Classic approach

In [138]:
M = np.matrix([[np.inf,2,3,5],
             [2,np.inf,6,1],
             [3,6,np.inf,4],
             [5,1,4,np.inf]])

rem_nodes = list(range(2,4+1))
cost = 0
optimal_cost = np.inf
v = [1]

# Note: the third variable optimal cost is used to determine if we have reached the end of a branch

def BnBClassic(A,v,rem_nodes, cost, optimal_cost):
    
    # End of the path
    if len(rem_nodes)==1:
        cost += A[v[-1]-1, rem_nodes[0]-1] + A[rem_nodes[0]-1,0]
        v += rem_nodes + [1]
        return cost, v, cost
    
    else:
        
        k = len(v)
        n = np.size(A,0)

        cost_branches = []
        path_branches = []
        f = []

        # Calculating the costs, the f and the paths
        for i in rem_nodes:
            cost_branches.append(cost + A[v[-1]-1,i-1])
            path_branches.append(v+[i])
            f.append(cost_branches[-1] + (n-k)*
                     np.min(A[path_branches[-1][-1]-1,[x-1 for x in rem_nodes if x!= k]]))

        # Sorting the arrays
        order = np.argsort(f)
        cost_branches = [cost_branches[i] for i in order]
        path_branches = [path_branches[i] for i in order]
        f = [f[i] for i in order]      


        # We explore recursively the branches and check if an optimal solution can be found
        for i in range(len(f)):
            # We discard all of the branches that cannot decrease the cost function
            # As all of the branches are sorted by cost, the following branches after
            # a discarded one will also be discarded
            if f[i] >= optimal_cost:
                break
            else:
                rem_nodes_sub = [x for x in rem_nodes if x not in path_branches[i]]
                cost, v, optimal_cost = BnBClassic(A,path_branches[i],rem_nodes_sub, cost_branches[i], optimal_cost)
            
            return cost, v, optimal_cost

t1 = time.time()     
    
cost, path, opt = BnBClassic(M,v,rem_nodes,cost,optimal_cost)

t2= time.time()

print('Time:', t2-t1)    
print('Minimum_cost:',cost)
print('Best path:',path)    

Time: 0.0010008811950683594
Minimum_cost: 10.0
Best path: [1, 2, 4, 3, 1]


## 4. Adding and removing edges

# DISREGARD THIS SECTION, UNDER CONSTRUCTION

In [130]:
# Helper functions used in the following algorithm

# This function calculates the normaliced matrix with at least one zero in each row and column.
# It also computes the cost of reduction.
def mat_redux(M, cost):
    
    # Used to reshape the minimum value arrays into matrices
    n = np.size(M,1)    
    
    # Reduction by rows
    m1 = M.min(axis=1)
    # to avoid inf-inf math error
    m1[m1 == np.inf] = 0
    Mrow = M - m1.reshape(n,1)
    
    # Reduction by columns
    m2 = Mrow.min(axis=0)
    m2[m2 == np.inf] = 0
    Mcol = Mrow - np.matrix(m2).reshape(1,n)
    
    # Lower bound calculation
    f = m1.sum() + m2.sum()
    
    return cost + f, Mcol

# This function selects the zero corresponding to the edge with the greatest weight
def select_edges(A, n):

    # We wish to know where are the pivots
    zeros_irow, zeros_icol = np.where(A==0)

    # Variables initialization
    max_edge_cost = 0
    opt_edge = (np.inf,np.inf)

    #  We traverse every zero such that it has the highest cost
    for k in range(len(zeros_irow)):
        
        i = zeros_irow[k]
        j = zeros_icol[k]
        x = [x for x in range(0,n) if x != i]
        y = [x for x in range(0,n) if x != j]

        edge_cost = np.min(A[i,y]) + np.min(A[x,j])   
        
        if edge_cost > max_edge_cost:
            
            max_edge_cost = edge_cost
            opt_edge = (i,j)

    return opt_edge

# This function created the children Left and Right subtree (with and without the edge)
def children_subtrees(M, edge, f):
    
    # Left and Right matrices
    L = np.copy(M)
    R = np.copy(M)

    # For the left child we add infinity to the column and row corresponding to the chosen edge
    L[edge[0],:] = np.inf
    L[:,edge[1]] = np.inf
    L[edge[1],edge[0]] = np.inf
    # After this the matrix should be reduced
    f_L, L = mat_redux(L,f)

    # For the right child we just add infinity to the location of the node, so that it's excluded
    R[edge[0], edge[1]] = np.inf
    f_R, R = mat_redux(R,f)

    return f_L, L, f_R, R

# Function used to reconstruct the path from the given edges 
# (the edge list is assumed to be same size of the number of cities)
def create_path_from_edges(E):
    
    # Default first city
    path = [1]
    
    while len(E) != 0:
        
        for i in range(0,len(E)):
            if E[i][0] == path[-1]-1:
                path.append(E[i][1]+1)
                E = E[0:i] + E[i+1:]
                break
                
    return path

In [ ]:
M = np.matrix([[np.inf, 3, 93, 13, 33, 9, 57],
               [4, np.inf, 77, 42, 21, 16, 34],
               [45, 17, np.inf, 36, 16, 28, 25],
               [39, 90, 80, np.inf, 56, 7, 91],
               [28, 46, 88, 33, np.inf, 25, 57],
               [3, 88, 18, 46, 92, np.inf, 7],
               [44, 26, 33, 27, 84, 39, np.inf]])

# M = np.matrix([[np.inf,2,3,5],
#              [2,np.inf,6,1],
#              [3,6,np.inf,4],
#              [5,1,4,np.inf]])

# The algorithm
def BnBBinaryTree(M):
    # Matrix reduction step
    f,MR = mat_redux(M,0)
    
    n = np.size(M,1)

    cost, edges_col = BnBBinaryTreeRecursive(MR,[],f,np.inf, n)
    
    return cost, create_path_from_edges(edges_col)

# This is the recursive algorithm that is called after pre-processing the matrix    
def BnBBinaryTreeRecursive(M, edges_collection, cost, opt_cost, n):
    
    print('The cost of salvation:',cost)
    print(edges_collection)
    
    # We know that we have found a solution if there are less nodes than cities to visit
    if n-len(edges_collection) > np.sum(M.min(axis=1) == 0):
        print('No solution possible, return infinite cost!')
        return np.inf, edges_collection
    
    # If the search is complete, 
    elif n-len(edges_collection) == np.sum(M.min(axis=1) == 0) == 0:
        print('Solution found!')
        if cost < opt_cost:
            return cost, edges_collection
        else:
            return opt_cost, edges_collection
    
    # Edge selection
    edge = select_edges(M, n)
    print(M)

    # Children trees creation
    f_L, L, f_R, R = children_subtrees(M, edge, cost)
    
    print(f_L)
    
    # We search the trees
    opt_cost, edges_col = BnBBinaryTreeRecursive(L, edges_collection + [edge], f_L, opt_cost, n)
    if opt_cost > f_R:
        opt_cost, edges_col = BnBBinaryTreeRecursive(R, edges_collection, f_R, opt_cost, n)
    
    print('cost L',opt_cost)
    print('cost R',opt_cost)
    
    print(edges_col)
    print(opt_cost)
    
    return opt_cost, edges_col
    
t1 = time.time()

cost, path = BnBBinaryTree(M)
    
t2= time.time()
print('Time:', t2-t1)    
print('Minimum_cost:',cost)
print('Best path:',path)
    

The cost of salvation: 96.0
[]
[[ inf   0.  83.   9.  30.   6.  50.]
 [  0.  inf  66.  37.  17.  12.  26.]
 [ 29.   1.  inf  19.   0.  12.   5.]
 [ 32.  83.  66.  inf  49.   0.  80.]
 [  3.  21.  56.   7.  inf   0.  28.]
 [  0.  85.   8.  42.  89.  inf   0.]
 [ 18.   0.   0.   0.  58.  13.  inf]]
99.0
The cost of salvation: 99.0
[(3, 5)]
[[ inf   0.  83.   9.  30.  inf  50.]
 [  0.  inf  66.  37.  17.  inf  26.]
 [ 29.   1.  inf  19.   0.  inf   5.]
 [ inf  inf  inf  inf  inf  inf  inf]
 [  0.  18.  53.   4.  inf  inf  25.]
 [  0.  85.   8.  inf  89.  inf   0.]
 [ 18.   0.   0.   0.  58.  inf  inf]]
99.0
The cost of salvation: 99.0
[(3, 5), (2, 4)]
[[ inf   0.  83.   9.  inf  inf  50.]
 [  0.  inf  66.  37.  inf  inf  26.]
 [ inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf]
 [  0.  18.  inf   4.  inf  inf  25.]
 [  0.  85.   8.  inf  inf  inf   0.]
 [ 18.   0.   0.   0.  inf  inf  inf]]
112.0
The cost of salvation: 112.0
[(3, 5), (2, 4), (1, 0)]
[[ inf  inf  74.

In [128]:
# M = np.matrix([[np.inf,2,3,5],
#              [2,np.inf,6,1],
#              [3,6,np.inf,4],
#              [5,1,4,np.inf]])

# M = np.matrix([[np.inf, 3, 93, 13, 33, 9],
#                [4, np.inf, 77, 42, 21, 16],
#                [45, 17, np.inf, 36, 16, 28],
#                [39, 90, 80, np.inf, 56, 7],
#                [28, 46, 88, 33, np.inf, 25],
#                [3, 88, 18, 46, 92, np.inf]])

M = np.matrix([[np.inf, 3, 93, 13, 33, 9, 57],
               [4, np.inf, 77, 42, 21, 16, 34],
               [45, 17, np.inf, 36, 16, 28, 25],
               [39, 90, 80, np.inf, 56, 7, 91],
               [28, 46, 88, 33, np.inf, 25, 57],
               [3, 88, 18, 46, 92, np.inf, 7],
               [44, 26, 33, 27, 84, 39, np.inf]])

# This function calculates the normaliced matrix with at least one zero in each row and column.
# It also computes the cost of reduction.
def mat_redux(M, cost):
    
    n = np.size(M,1)    
    
    m1 = M.min(axis=1)
    # to avoid inf-inf math error
    m1[m1 == np.inf] = 0
    Mrow = M - m1.reshape(n,1)
    m2 = Mrow.min(axis=0)
    m2[m2 == np.inf] = 0
    Mcol = Mrow - np.matrix(m2).reshape(1,n)
    f = m1.sum() + m2.sum()
    
    return cost + f, Mcol

def select_edges(A):

    zeros_irow, zeros_icol = np.where(A==0)

    n = np.size(A,1)

    max_edge_cost = 0
    opt_edges = (np.inf,np.inf)

    for k in range(len(zeros_irow)):
        i = zeros_irow[k]
        j = zeros_icol[k]
        x = [x for x in range(0,n) if x != i]
        y = [x for x in range(0,n) if x != j]

        edge_cost = np.min(A[i,y]) + np.min(A[x,j])   
        
        if edge_cost > max_edge_cost:

            max_edge_cost = edge_cost
            opt_edges = (i,j)

            
    print(A)
    return max_edge_cost, opt_edges    
    
    
f,MR = mat_redux(M,0)
print(MR)
print(f)

edge_cost,edge = select_edges(MR)
print(edge)

# Create children
def children_subtrees(M, edge, f):
    L = np.copy(M)
    R = np.copy(M)

    L[edge[0],:] = np.inf
    L[:,edge[1]] = np.inf
    L[edge[1],edge[0]] = np.inf
    f_L, L = mat_redux(L,f)

    R[edge[0], edge[1]] = np.inf
    f_R, R = mat_redux(R,f)

    return f_L, L, f_R, R

f_L, L, f_R, R = children_subtrees(MR, edge, f)

edges_collection = [edge]

print('----------------------')
print('Children')
print('L:')
print(L)
print('cost L:', f_L)
print('R:')
print(R)
print('cost R:', f_R)


for i in range(1,7):
    edge_cost,edge = select_edges(L)
    f_L, L, f_R, R = children_subtrees(L, edge, f_L)
    edges_collection.append(edge)
    print('for', i)
    print('L:')
    print(edges_collection)
    print(L)
    print('R:')
    print(edges_collection[:-1])
    print(R)
    print('cost L:', f_L)
    print('cost R:', f_R)
    

# Function used to create the path from the obtained edges
def create_path_from_edges(E):
    
    path = [1]
    
    while len(E) != 0:
        
        for i in range(0,len(E)):
            if E[i][0] == path[-1]-1:
                path.append(E[i][1]+1)
                E = E[0:i] + E[i+1:]
                break
                
    return path

# We know that we have found a solution if there are less nodes than cities to visit
if n-len(edges_collection) > np.sum(L.min(axis=1) == 0):
    print('No solution possible, return infinite cost!')
elif n-len(edges_collection) == np.sum(L.min(axis=1) == 0) == 0:
    print('Solution found!')
else:
    print('Keep on searching')

[[ inf   0.  83.   9.  30.   6.  50.]
 [  0.  inf  66.  37.  17.  12.  26.]
 [ 29.   1.  inf  19.   0.  12.   5.]
 [ 32.  83.  66.  inf  49.   0.  80.]
 [  3.  21.  56.   7.  inf   0.  28.]
 [  0.  85.   8.  42.  89.  inf   0.]
 [ 18.   0.   0.   0.  58.  13.  inf]]
96.0
[[ inf   0.  83.   9.  30.   6.  50.]
 [  0.  inf  66.  37.  17.  12.  26.]
 [ 29.   1.  inf  19.   0.  12.   5.]
 [ 32.  83.  66.  inf  49.   0.  80.]
 [  3.  21.  56.   7.  inf   0.  28.]
 [  0.  85.   8.  42.  89.  inf   0.]
 [ 18.   0.   0.   0.  58.  13.  inf]]
(3, 5)
----------------------
Children
L:
[[ inf   0.  83.   9.  30.  inf  50.]
 [  0.  inf  66.  37.  17.  inf  26.]
 [ 29.   1.  inf  19.   0.  inf   5.]
 [ inf  inf  inf  inf  inf  inf  inf]
 [  0.  18.  53.   4.  inf  inf  25.]
 [  0.  85.   8.  inf  89.  inf   0.]
 [ 18.   0.   0.   0.  58.  inf  inf]]
cost L: 99.0
R:
[[ inf   0.  83.   9.  30.   6.  50.]
 [  0.  inf  66.  37.  17.  12.  26.]
 [ 29.   1.  inf  19.   0.  12.   5.]
 [  0.  51.  34.  inf 

In [75]:
edges_collection

n = np.size(M,1)

print(len(edges_collection))
print(n-len(edges_collection[:-1]))
print(np.sum(L.min(axis=1) == 0))

6
2
1
